In [1]:
import os

# start ss 
ss setup
need to export these in shell to run poetry in shell

* `export PINECONE_ENVIRONMENT="us-west1-gcp"`
* `export PINECONE_INDEX="tkcrfp"`
* ` poetry run start`

In [2]:
# need to export these in shell to run poetry in shell
os.environ["PINECONE_ENVIRONMENT"]="us-west1-gcp"
os.environ["PINECONE_INDEX"]="tkcrfp"


In [ ]:
from langchain.document_loaders import DirectoryLoader